# Complete API Testing Flow - Ziya LRG API

This notebook provides a comprehensive testing flow for all API endpoints in the Ziya LRG platform.

## API Categories Covered:
1. **Health & Meta** - Server health checks and endpoint discovery
2. **Content Management** - Reading, Listening, Grammar, Writing, Speaking
3. **Session Management** - Starting and submitting test sessions
4. **Listening Evaluation** - Specialized listening test sessions
5. **XP & Streaks** - Experience points and streak tracking
6. **Skill Mastery** - Skill progress and competency tracking
7. **Dashboard** - Analytics and user progress
8. **Unified Analytics** - Combined LRG, Writing, Speaking data

## Test Flow Strategy:
- Each section builds upon previous sections
- Proper setup and cleanup
- Error handling to prevent test failures
- Visual output for easy verification

## 1. Setup & Configuration

In [3]:
import requests
import json
from datetime import datetime, timedelta, timezone
from uuid import uuid4
import time
from typing import Optional, Dict, Any
import jwt
# API Configuration
BASE_URL = "https://zia-lrg-166647007319.europe-west1.run.app/api/v1"
ROOT_URL = "https://zia-lrg-166647007319.europe-west1.run.app"

# Test User Configuration
TEST_USER_ID = "123e4567-e89b-12d3-a456-426614174000"  # Replace with actual user UUID
AUTH_TOKEN = "test-token-12345"  # Replace with actual auth token
TEST_USER_ID = str(uuid4())  # Replace with actual user_id if needed

# Create a JWT token for authentication (matches what the API expects)
def create_test_token(user_id: str) -> str:
    """Create a test JWT token with user_id in 'sub' claim"""
    payload = {
        "sub": user_id,  # User ID
        "iat": datetime.now(timezone.utc),
        "exp": datetime.now(timezone.utc) + timedelta(hours=24)
    }
    # Note: API uses verify_signature=False, so any token with correct structure works
    token = jwt.encode(payload, "test-secret", algorithm="HS256")
    return token

# Generate token
AUTH_TOKEN = create_test_token(TEST_USER_ID)

# Headers for authenticated requests
AUTH_HEADERS = {
    "Authorization": f"Bearer {AUTH_TOKEN}",
    "Content-Type": "application/json"
}
# Request Headers
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {AUTH_TOKEN}"
}

# Color codes for output
class Colors:
    GREEN = '\033[92m'
    RED = '\033[91m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    RESET = '\033[0m'

# Helper function to print formatted responses
def print_response(title: str, response: requests.Response, show_body: bool = True):
    """Print formatted API response"""
    status_color = Colors.GREEN if response.status_code < 300 else Colors.YELLOW if response.status_code < 400 else Colors.RED
    
    print(f"\n{Colors.BLUE}{'='*80}{Colors.RESET}")
    print(f"{Colors.BLUE}{title}{Colors.RESET}")
    print(f"{status_color}Status: {response.status_code}{Colors.RESET}")
    
    if show_body:
        try:
            print(f"\n{Colors.YELLOW}Response:{Colors.RESET}")
            print(json.dumps(response.json(), indent=2))
        except:
            print(f"Response: {response.text}")
    
    print(f"{Colors.BLUE}{'='*80}{Colors.RESET}")
    return response

# Helper function for safe API calls
def safe_api_call(method: str, url: str, **kwargs) -> Optional[requests.Response]:
    """Make API call with error handling"""
    try:
        response = requests.request(method, url, **kwargs)
        return response
    except Exception as e:
        print(f"{Colors.RED}Error: {e}{Colors.RESET}")
        return None

# Storage for test data
test_data = {
    'reading_id': None,
    'listening_id': None,
    'grammar_id': None,
    'writing_id': None,
    'speaking_id': None,
    'session_id': None,
    'listening_session_id': None
}

print(f"{Colors.GREEN}Setup Complete!{Colors.RESET}")
print(f"Base URL: {BASE_URL}")
print(f"Test User ID: {TEST_USER_ID}")

Setup Complete!
Base URL: https://zia-lrg-166647007319.europe-west1.run.app/api/v1
Test User ID: 87f27677-c00e-43d8-abc1-9ee77ef508e5


## 2. Health & Meta Endpoints

In [5]:
# Test root endpoint
response = safe_api_call("GET", f"{ROOT_URL}/")
if response:
    print_response("Root Endpoint - Health Check", response)

# Test health endpoint
response = safe_api_call("GET", f"{ROOT_URL}/health")
if response:
    print_response("Health Check - Detailed", response)

# List all API endpoints
response = safe_api_call("GET", f"{BASE_URL}/meta/endpoints")
if response:
    print_response("Meta - List All Endpoints", response, show_body=False)
    if response.status_code == 200:
        data = response.json()
        print(f"\n{Colors.GREEN}Total Endpoints: {data.get('count', 0)}{Colors.RESET}")
        print(f"\nEndpoint Categories:")
        tags = {}
        for route in data.get('routes', []):
            for tag in route.get('tags', ['Uncategorized']):
                tags[tag] = tags.get(tag, 0) + 1
        for tag, count in sorted(tags.items()):
            print(f"  {tag}: {count} endpoints")


Root Endpoint - Health Check
Status: 200

Response:
{
  "status": "healthy",
  "service": "Ziya LRG API",
  "version": "1.0.0"
}

Health Check - Detailed
Status: 200

Response:
{
  "status": "healthy",
  "database": "connected",
  "storage": "ready"
}

Meta - List All Endpoints
Status: 200

Total Endpoints: 56

Endpoint Categories:
  Content: 25 endpoints
  Dashboard: 3 endpoints
  Listening Evaluation: 6 endpoints
  Meta: 1 endpoints
  Sessions: 4 endpoints
  Skill Mastery: 4 endpoints
  Unified Analytics: 7 endpoints
  XP & Streaks: 6 endpoints


## 3. Content Management - Reading

In [6]:
# Create Reading Content
reading_data = {
    "day_code": "day1",
    "difficulty_level": "beginner",
    "title": "Introduction to Python",
    "passage": "Python is a high-level, interpreted programming language known for its simplicity and readability.",
    "questions": [
        {
            "question_text": "What type of language is Python?",
            "question_type": "multiple_choice",
            "options": ["High-level", "Low-level", "Assembly", "Machine"],
            "correct_answer": "High-level",
            "skill_tag": "comprehension"
        },
        {
            "question_text": "Python is known for its _____ and _____.",
            "question_type": "fill_blank",
            "correct_answer": "simplicity, readability",
            "skill_tag": "vocabulary"
        }
    ]
}

response = safe_api_call("POST", f"{BASE_URL}/reading", headers=HEADERS, json=reading_data)
if response:
    print_response("Create Reading Content", response)
    if response.status_code == 201:
        test_data['reading_id'] = response.json().get('reading_id')
        print(f"{Colors.GREEN}Reading ID saved: {test_data['reading_id']}{Colors.RESET}")

# Get Reading by Day
response = safe_api_call("GET", f"{BASE_URL}/reading/day/day1", headers=HEADERS)
if response:
    print_response("Get Reading by Day (day1)", response)

# Get Reading by ID (if we have one)
if test_data['reading_id']:
    response = safe_api_call("GET", f"{BASE_URL}/reading/{test_data['reading_id']}", headers=HEADERS)
    if response:
        print_response(f"Get Reading by ID ({test_data['reading_id']})", response)

# Update Reading (if we have one)
if test_data['reading_id']:
    update_data = reading_data.copy()
    update_data['title'] = "Introduction to Python - Updated"
    response = safe_api_call("PUT", f"{BASE_URL}/reading/{test_data['reading_id']}", headers=HEADERS, json=update_data)
    if response:
        print_response("Update Reading Content", response)


Get Reading by Day (day1)
Status: 200

Response:
{
  "day_code": "day1",
  "readings": [
    {
      "reading_id": "e95b3402-e8dc-422a-8ff1-f33247477224",
      "day_code": "day1",
      "title": " The Missing Context",
      "passage": "Story: Jian, a software developer in Singapore, sent a terse, one-line email to his team in Manila: \"Database update needed urgently.\" He assumed the technical context was obvious. However, his team was working on a different priority and interpreted \"urgently\" as needing to drop everything immediately, causing unnecessary panic. The project manager intervened, advising Jian: \"When communicating across cultures and teams, clarity trumps brevity. Always include the 'why' and the 'impact'.\" Jian learned that the effort saved by typing less was offset by the time lost to confusion and misinterpretation.\n\nMoral: Clarity in communication, especially across diverse teams, requires context and explicit explanation of the urgency and impact.\n",
     

## 4. Content Management - Grammar

In [7]:
# Create Grammar Content
grammar_data = {
    "day_code": "day1",
    "difficulty_level": "beginner",
    "topic": "Present Simple Tense",
    "explanation": "The present simple tense is used to describe habits, unchanging situations, general truths, and fixed arrangements.",
    "examples": [
        "I work in an office.",
        "She plays tennis every weekend.",
        "The sun rises in the east."
    ],
    "questions": [
        {
            "question_text": "Choose the correct form: He _____ to work every day.",
            "question_type": "multiple_choice",
            "options": ["go", "goes", "going", "went"],
            "correct_answer": "goes",
            "skill_tag": "verb_conjugation"
        },
        {
            "question_text": "They _____ (play) football on Saturdays.",
            "question_type": "fill_blank",
            "correct_answer": "play",
            "skill_tag": "verb_conjugation"
        }
    ]
}

response = safe_api_call("POST", f"{BASE_URL}/grammar", headers=HEADERS, json=grammar_data)
if response:
    print_response("Create Grammar Content", response)
    if response.status_code == 201:
        test_data['grammar_id'] = response.json().get('grammar_id')
        print(f"{Colors.GREEN}Grammar ID saved: {test_data['grammar_id']}{Colors.RESET}")

# Get Grammar by Day
response = safe_api_call("GET", f"{BASE_URL}/grammar/day/day1", headers=HEADERS)
if response:
    print_response("Get Grammar by Day (day1)", response)

# Get Grammar by ID
if test_data['grammar_id']:
    response = safe_api_call("GET", f"{BASE_URL}/grammar/{test_data['grammar_id']}", headers=HEADERS)
    if response:
        print_response(f"Get Grammar by ID ({test_data['grammar_id']})", response)


Get Grammar by Day (day1)
Status: 200

Response:
{
  "day_code": "day1",
  "grammar_sets": [
    {
      "grammar_id": "138ef5a2-d86c-40a4-ba3f-cd0bb3ab8eec",
      "day_code": "day1",
      "title": "ADVANCED GRAMMAR AND NUANCED USAGE",
      "tasks": [
        {
          "id": "g1760003940771",
          "type": "mcq",
          "prompt": "Had I known you were coming, I ___________ a meal for you.",
          "options": {
            "a": "would prepare",
            "b": "will prepare",
            "c": "would have prepared",
            "d": ""
          },
          "answer": "c",
          "explanation": "",
          "topic": ""
        },
        {
          "id": "g1760003942211",
          "type": "mcq",
          "prompt": "She _________ to the store, but I'm not sure.",
          "options": {
            "a": "might have gone",
            "b": "might go",
            "c": "must go ",
            "d": ""
          },
          "answer": "a",
          "explanation": "",
 

## 5. Content Management - Writing

In [8]:
# Create Writing Content
writing_data = {
    "day_code": "day1",
    "difficulty_level": "beginner",
    "prompt": "Write a short paragraph about your favorite hobby.",
    "topic": "Hobbies and Interests",
    "word_limit": 150,
    "rubric": {
        "grammar": "Check for proper sentence structure and tense usage",
        "vocabulary": "Use appropriate hobby-related vocabulary",
        "organization": "Clear introduction and conclusion",
        "creativity": "Original ideas and personal examples"
    },
    "sample_response": "My favorite hobby is reading. I enjoy reading because it allows me to explore new worlds and learn about different cultures. Every evening, I spend at least one hour reading books. This hobby helps me relax and expand my knowledge."
}

response = safe_api_call("POST", f"{BASE_URL}/writing", headers=HEADERS, json=writing_data)
if response:
    print_response("Create Writing Content", response)
    if response.status_code == 201:
        test_data['writing_id'] = response.json().get('writing_id')
        print(f"{Colors.GREEN}Writing ID saved: {test_data['writing_id']}{Colors.RESET}")

# Get Writing by Day
response = safe_api_call("GET", f"{BASE_URL}/writing/day/day1", headers=HEADERS)
if response:
    print_response("Get Writing by Day (day1)", response)

# Get Writing by ID
if test_data['writing_id']:
    response = safe_api_call("GET", f"{BASE_URL}/writing/{test_data['writing_id']}", headers=HEADERS)
    if response:
        print_response(f"Get Writing by ID ({test_data['writing_id']})", response)


Get Writing by Day (day1)
Status: 200

Response:
{
  "day_code": "day1",
  "writings": [],
  "count": 0
}


## 6. Content Management - Speaking

In [9]:
# Create Speaking Content
speaking_data = {
    "day_code": "day1",
    "difficulty_level": "beginner",
    "teaching_mode_code": "conversation",
    "topic": "Introducing Yourself",
    "prompt": "Introduce yourself to a new colleague. Include your name, job, and a hobby.",
    "context": "You are at a company meeting and meeting a new team member for the first time.",
    "duration_sec": 60,
    "evaluation_criteria": {
        "pronunciation": "Clear and understandable pronunciation",
        "fluency": "Smooth speech without excessive pauses",
        "vocabulary": "Appropriate workplace vocabulary",
        "grammar": "Correct use of present tense"
    },
    "sample_phrases": [
        "Hello, my name is...",
        "I work as a...",
        "In my free time, I enjoy...",
        "Nice to meet you!"
    ]
}

response = safe_api_call("POST", f"{BASE_URL}/speaking", headers=HEADERS, json=speaking_data)
if response:
    print_response("Create Speaking Content", response)
    if response.status_code == 201:
        test_data['speaking_id'] = response.json().get('speaking_id')
        print(f"{Colors.GREEN}Speaking ID saved: {test_data['speaking_id']}{Colors.RESET}")

# Get Speaking by Day
response = safe_api_call("GET", f"{BASE_URL}/speaking/day/day1", headers=HEADERS)
if response:
    print_response("Get Speaking by Day (day1)", response)

# Get Speaking by ID
if test_data['speaking_id']:
    response = safe_api_call("GET", f"{BASE_URL}/speaking/{test_data['speaking_id']}", headers=HEADERS)
    if response:
        print_response(f"Get Speaking by ID ({test_data['speaking_id']})", response)


Get Speaking by Day (day1)
Status: 200

Response:
{
  "day_code": "day1",
  "topics": [
    {
      "speaking_id": "1deb3244-9d8b-48d1-9d73-b11453b757e8",
      "day_code": "day1",
      "teaching_mode_id": "c7c4631e-8e22-40e8-9ee9-c48936bc799b",
      "teaching_mode_code": "grammar",
      "title": "Advanced",
      "topic": "Let\u2019s Talk About Your Favorite Sportsperson",
      "context": " (Focus on their great achievements, how hard they practice, and what you learn from them.)\n",
      "difficulty_level": "advanced",
      "metadata": {
        "uploaded_by": "admin@ailt.in",
        "company_name": "AILT",
        "uploaded_time": "2025-10-07T11:34:04.089000+00:00"
      },
      "created_at": "2025-10-07T11:34:03.736848Z"
    },
    {
      "speaking_id": "179ed83a-83ea-4c95-a883-32bcb39859d9",
      "day_code": "day1",
      "teaching_mode_id": "c7c4631e-8e22-40e8-9ee9-c48936bc799b",
      "teaching_mode_code": "grammar",
      "title": "MEDIUM",
      "topic": "The Import

## 7. Session Management - LRG Sessions

In [10]:
# Start a Session
session_start_data = {
    "user_id": TEST_USER_ID,
    "modality": "reading",
    "day_code": "day1"
}

response = safe_api_call("POST", f"{BASE_URL}/sessions", headers=HEADERS, json=session_start_data)
if response:
    print_response("Start Session", response)
    if response.status_code == 201:
        test_data['session_id'] = response.json().get('session_id')
        print(f"{Colors.GREEN}Session ID saved: {test_data['session_id']}{Colors.RESET}")

# Submit Session (if we have a session_id)
if test_data['session_id']:
    session_submit_data = {
        "answers": [
            {
                "question_id": str(uuid4()),
                "user_answer": "High-level",
                "is_correct": True,
                "time_spent_sec": 15,
                "skill_tag": "comprehension"
            },
            {
                "question_id": str(uuid4()),
                "user_answer": "simplicity, readability",
                "is_correct": True,
                "time_spent_sec": 20,
                "skill_tag": "vocabulary"
            }
        ],
        "duration_sec": 35,
        "score_pct": 100.0
    }
    
    response = safe_api_call("POST", f"{BASE_URL}/sessions/{test_data['session_id']}/submit", 
                            headers=HEADERS, json=session_submit_data)
    if response:
        print_response("Submit Session", response)

# Get Session Details
if test_data['session_id']:
    response = safe_api_call("GET", f"{BASE_URL}/sessions/{test_data['session_id']}", headers=HEADERS)
    if response:
        print_response("Get Session Details", response)

# Get User Sessions List
response = safe_api_call("GET", f"{BASE_URL}/sessions?limit=5&offset=0", headers=HEADERS)
if response:
    print_response("Get User Sessions (Paginated)", response)


Get User Sessions (Paginated)
Status: 200

Response:
{
  "sessions": [],
  "limit": 5,
  "offset": 0,
  "count": 0
}


## 8. Listening Evaluation Sessions

In [11]:
# Start Listening Session
listening_session_data = {
    "user_id": TEST_USER_ID,
    "day_code": "day1",
    "audio_url": "https://example.com/audio/day1.mp3"
}

response = safe_api_call("POST", f"{BASE_URL}/listening/sessions", headers=HEADERS, json=listening_session_data)
if response:
    print_response("Start Listening Session", response)
    if response.status_code == 201:
        test_data['listening_session_id'] = response.json().get('session_id')
        print(f"{Colors.GREEN}Listening Session ID saved: {test_data['listening_session_id']}{Colors.RESET}")

# Submit Listening Session
if test_data['listening_session_id']:
    listening_submit_data = {
        "answers": [
            {
                "question_id": str(uuid4()),
                "user_answer": "Paris",
                "is_correct": True,
                "time_spent_sec": 10,
                "skill_tag": "main_idea",
                "audio_replay_count": 2,
                "time_to_first_answer_sec": 5
            },
            {
                "question_id": str(uuid4()),
                "user_answer": "tomorrow",
                "is_correct": True,
                "time_spent_sec": 12,
                "skill_tag": "detail",
                "audio_replay_count": 1,
                "time_to_first_answer_sec": 8
            }
        ],
        "duration_sec": 45,
        "score_pct": 100.0,
        "xp_earned": 50,
        "audio_replay_count": 3,
        "completed_at": datetime.utcnow().isoformat() + "Z"
    }
    
    response = safe_api_call("POST", f"{BASE_URL}/listening/sessions/{test_data['listening_session_id']}/submit",
                            headers=HEADERS, json=listening_submit_data)
    if response:
        print_response("Submit Listening Session", response)

# Get Listening Session Details
if test_data['listening_session_id']:
    response = safe_api_call("GET", f"{BASE_URL}/listening/sessions/{test_data['listening_session_id']}",
                            headers=HEADERS)
    if response:
        print_response("Get Listening Session Details", response)

# Get Listening Session Mastery
if test_data['listening_session_id']:
    response = safe_api_call("GET", f"{BASE_URL}/listening/sessions/{test_data['listening_session_id']}/mastery",
                            headers=HEADERS)
    if response:
        print_response("Get Listening Session Mastery Breakdown", response)

# Get User Listening Progress
response = safe_api_call("GET", f"{BASE_URL}/listening/users/{TEST_USER_ID}/progress", headers=HEADERS)
if response:
    print_response("Get User Listening Progress", response)

# Get Listening Analytics
response = safe_api_call("GET", f"{BASE_URL}/listening/users/{TEST_USER_ID}/analytics", headers=HEADERS)
if response:
    print_response("Get Listening Analytics", response)


Get User Listening Progress
Status: 200

Response:
{
  "modality": "listening",
  "date_range": "day1-latest",
  "overall_mastery_pct": 0,
  "total_sessions": 0,
  "total_audio_replay_count": 0,
  "skills": []
}


## 9. XP & Streaks

In [12]:
# Get User XP Summary
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/xp", headers=HEADERS)
if response:
    print_response("Get User XP Summary", response)

# Get Daily XP
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/xp/daily", headers=HEADERS)
if response:
    print_response("Get Daily XP Breakdown", response)

# Get User Level
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/level", headers=HEADERS)
if response:
    print_response("Get User Level Information", response)

# Get User Streak
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/streak", headers=HEADERS)
if response:
    print_response("Get User Streak Information", response)

# Get Daily Progress
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/daily-progress", headers=HEADERS)
if response:
    print_response("Get Daily Progress", response)

# Get Streak Calendar
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/streak-calendar", headers=HEADERS)
if response:
    print_response("Get Streak Calendar (Current Month)", response)

# Get Streak Calendar for Specific Month
current_month = datetime.now().strftime("%Y-%m")
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/streak-calendar?month={current_month}",
                        headers=HEADERS)
if response:
    print_response(f"Get Streak Calendar ({current_month})", response)


Get User XP Summary
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-abc1-9ee77ef508e5",
  "total_xp": 0,
  "today_xp": 0,
  "current_level": 1,
  "xp_to_next_level": 150,
  "level_progress_pct": 0
}

Get Daily XP Breakdown
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-abc1-9ee77ef508e5",
  "date": "2025-10-18",
  "xp_earned_today": 0,
  "xp_goal": 100,
  "goal_completion_pct": 0,
  "sessions_today": 0,
  "breakdown": []
}

Get User Level Information
Status: 200

Response:
{
  "current_level": 1,
  "level_name": "Beginner",
  "total_xp": 0,
  "xp_for_current_level": 0,
  "xp_for_next_level": 150,
  "xp_to_next_level": 150,
  "progress_pct": 0
}

Get User Streak Information
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-abc1-9ee77ef508e5",
  "current_streak": 0,
  "longest_streak": 0,
  "last_active_date": "2025-10-18",
  "is_active_today": false,
  "streak_status": "broken"
}

Get Daily Progress
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-

## 10. Skill Mastery & Competencies

In [13]:
# Get Session Mastery (if we have a session)
if test_data['session_id']:
    response = safe_api_call("GET", f"{BASE_URL}/sessions/{test_data['session_id']}/mastery", headers=HEADERS)
    if response:
        print_response("Get Session Skill Mastery Breakdown", response)

# Get User Skill Progress for Reading
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/skills/progress?modality=reading",
                        headers=HEADERS)
if response:
    print_response("Get User Skill Progress - Reading", response)

# Get User Skill Progress for Listening
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/skills/progress?modality=listening",
                        headers=HEADERS)
if response:
    print_response("Get User Skill Progress - Listening", response)

# Get User Skill Progress for Grammar
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/skills/progress?modality=grammar",
                        headers=HEADERS)
if response:
    print_response("Get User Skill Progress - Grammar", response)

# Get Mastery Overview (All Modalities)
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/mastery-overview", headers=HEADERS)
if response:
    print_response("Get Complete Mastery Overview", response)

# Get Competencies by Day
response = safe_api_call("GET", f"{BASE_URL}/users/{TEST_USER_ID}/competencies/reading/day/day1",
                        headers=HEADERS)
if response:
    print_response("Get Competencies - Reading Day 1", response)


Get User Skill Progress - Reading
Status: 200

Response:
{
  "modality": "reading",
  "date_range": "day1-latest",
  "skills": []
}

Get User Skill Progress - Listening
Status: 200

Response:
{
  "modality": "listening",
  "date_range": "day1-latest",
  "skills": []
}

Get User Skill Progress - Grammar
Status: 200

Response:
{
  "modality": "grammar",
  "date_range": "day1-latest",
  "skills": []
}

Get Complete Mastery Overview
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-abc1-9ee77ef508e5",
  "listening": {
    "overall_mastery_pct": 0,
    "skills": {}
  },
  "reading": {
    "overall_mastery_pct": 0,
    "skills": {}
  },
  "grammar": {
    "overall_mastery_pct": 0,
    "skills": {}
  }
}

Get Competencies - Reading Day 1
Status: 200

Response:
{
  "modality": "reading",
  "date_range": "day1",
  "skills": []
}


## 11. Dashboard Analytics

In [14]:
# Get Dashboard Summary (7 days)
response = safe_api_call("GET", f"{BASE_URL}/dashboard/summary?window=7d", headers=HEADERS)
if response:
    print_response("Dashboard Summary - 7 Days", response)

# Get Dashboard Summary (30 days)
response = safe_api_call("GET", f"{BASE_URL}/dashboard/summary?window=30d", headers=HEADERS)
if response:
    print_response("Dashboard Summary - 30 Days", response)

# Get Dashboard Summary (90 days)
response = safe_api_call("GET", f"{BASE_URL}/dashboard/summary?window=90d", headers=HEADERS)
if response:
    print_response("Dashboard Summary - 90 Days", response)

# Get Modality Detail - Listening
response = safe_api_call("GET", f"{BASE_URL}/dashboard/detail/listening", headers=HEADERS)
if response:
    print_response("Modality Detail - Listening", response)

# Get Modality Detail - Reading
response = safe_api_call("GET", f"{BASE_URL}/dashboard/detail/reading", headers=HEADERS)
if response:
    print_response("Modality Detail - Reading", response)

# Get Modality Detail - Grammar
response = safe_api_call("GET", f"{BASE_URL}/dashboard/detail/grammar", headers=HEADERS)
if response:
    print_response("Modality Detail - Grammar", response)

# Get User Progress
response = safe_api_call("GET", f"{BASE_URL}/dashboard/progress", headers=HEADERS)
if response:
    print_response("User Overall Progress", response)


Dashboard Summary - 7 Days
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-abc1-9ee77ef508e5",
  "as_of": "2025-10-18T12:45:50.380868",
  "streak_days": 0,
  "xp": {
    "total": 0,
    "today": 0
  },
  "daily_target": {
    "target": 3,
    "completed": 0
  },
  "last_activity": null,
  "weekly_minutes": [
    {
      "date": "2025-10-11",
      "listening_min": 0,
      "reading_min": 0,
      "grammar_min": 0
    },
    {
      "date": "2025-10-12",
      "listening_min": 0,
      "reading_min": 0,
      "grammar_min": 0
    },
    {
      "date": "2025-10-13",
      "listening_min": 0,
      "reading_min": 0,
      "grammar_min": 0
    },
    {
      "date": "2025-10-14",
      "listening_min": 0,
      "reading_min": 0,
      "grammar_min": 0
    },
    {
      "date": "2025-10-15",
      "listening_min": 0,
      "reading_min": 0,
      "grammar_min": 0
    },
    {
      "date": "2025-10-16",
      "listening_min": 0,
      "reading_min": 0,
      "grammar_min": 0
  

## 12. Unified Analytics

In [15]:
# Get Unified Stats
response = safe_api_call("GET", f"{BASE_URL}/unified/stats", headers=HEADERS)
if response:
    print_response("Unified Stats - All Activities", response)

# Get Unified Dashboard (7 days)
response = safe_api_call("GET", f"{BASE_URL}/unified/dashboard?days=7", headers=HEADERS)
if response:
    print_response("Unified Dashboard - 7 Days", response)

# Get Unified Dashboard (30 days)
response = safe_api_call("GET", f"{BASE_URL}/unified/dashboard?days=30", headers=HEADERS)
if response:
    print_response("Unified Dashboard - 30 Days", response)

# Get Activity Timeline
response = safe_api_call("GET", f"{BASE_URL}/unified/timeline?limit=10&offset=0", headers=HEADERS)
if response:
    print_response("Activity Timeline - Recent 10", response)

# Get Comprehensive Progress - Weekly
response = safe_api_call("GET", f"{BASE_URL}/unified/progress?period=weekly", headers=HEADERS)
if response:
    print_response("Comprehensive Progress - Weekly", response)

# Get Comprehensive Progress - Monthly
response = safe_api_call("GET", f"{BASE_URL}/unified/progress?period=monthly", headers=HEADERS)
if response:
    print_response("Comprehensive Progress - Monthly", response)

# Get Comprehensive Progress - All Time
response = safe_api_call("GET", f"{BASE_URL}/unified/progress?period=all_time", headers=HEADERS)
if response:
    print_response("Comprehensive Progress - All Time", response)

# Get Week Comparison
response = safe_api_call("GET", f"{BASE_URL}/unified/comparison", headers=HEADERS)
if response:
    print_response("Week-over-Week Comparison", response)

# Get Achievements Summary
response = safe_api_call("GET", f"{BASE_URL}/unified/achievements", headers=HEADERS)
if response:
    print_response("Achievements Summary", response)

# Get Learning Insights
response = safe_api_call("GET", f"{BASE_URL}/unified/insights", headers=HEADERS)
if response:
    print_response("Learning Insights & Recommendations", response)


Unified Stats - All Activities
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-abc1-9ee77ef508e5",
  "display_name": null,
  "email": null,
  "skill_level": null,
  "target_language": null,
  "lrg_sessions": 0,
  "lrg_time_sec": 0,
  "lrg_avg_score": null,
  "writing_evaluations": 0,
  "writing_avg_score": null,
  "speaking_sessions": 0,
  "current_streak": 0,
  "longest_streak": 0,
  "total_xp": 0,
  "last_activity": null
}

Unified Dashboard - 7 Days
Status: 200

Response:
{
  "user_id": "87f27677-c00e-43d8-abc1-9ee77ef508e5",
  "as_of": "2025-10-18T12:46:09.740654",
  "period_days": 7,
  "total_activities": 0,
  "total_time_minutes": 0,
  "current_streak": 0,
  "total_xp": 0,
  "lrg_stats": {
    "session_count": 0,
    "total_time_sec": 0,
    "avg_score": null,
    "by_modality": {
      "reading": 0,
      "listening": 0,
      "grammar": 0
    }
  },
  "writing_stats": {
    "evaluation_count": 0,
    "avg_score": null,
    "total_time_sec": 0,
    "recent_feedback": 

## 13. Cleanup (Optional)

In [ ]:
# Delete created test data (optional - uncomment to use)

# Delete Reading
# if test_data['reading_id']:
#     response = safe_api_call("DELETE", f"{BASE_URL}/reading/{test_data['reading_id']}", headers=HEADERS)
#     if response and response.status_code == 204:
#         print(f"{Colors.GREEN}Reading deleted successfully{Colors.RESET}")

# Delete Grammar
# if test_data['grammar_id']:
#     response = safe_api_call("DELETE", f"{BASE_URL}/grammar/{test_data['grammar_id']}", headers=HEADERS)
#     if response and response.status_code == 204:
#         print(f"{Colors.GREEN}Grammar deleted successfully{Colors.RESET}")

# Delete Writing
# if test_data['writing_id']:
#     response = safe_api_call("DELETE", f"{BASE_URL}/writing/{test_data['writing_id']}", headers=HEADERS)
#     if response and response.status_code == 204:
#         print(f"{Colors.GREEN}Writing deleted successfully{Colors.RESET}")

# Delete Speaking
# if test_data['speaking_id']:
#     response = safe_api_call("DELETE", f"{BASE_URL}/speaking/{test_data['speaking_id']}", headers=HEADERS)
#     if response and response.status_code == 204:
#         print(f"{Colors.GREEN}Speaking deleted successfully{Colors.RESET}")

print(f"{Colors.BLUE}Cleanup section (commented out by default){Colors.RESET}")
print(f"Uncomment delete operations if you want to clean up test data")

## 14. Test Summary

In [16]:
# Print test summary
print(f"\n{Colors.BLUE}{'='*80}{Colors.RESET}")
print(f"{Colors.GREEN}API TESTING COMPLETE{Colors.RESET}")
print(f"{Colors.BLUE}{'='*80}{Colors.RESET}\n")

print(f"{Colors.YELLOW}Test Data Created:{Colors.RESET}")
for key, value in test_data.items():
    status = Colors.GREEN + "✓" if value else Colors.RED + "✗"
    print(f"{status} {key}: {value if value else 'Not created'}{Colors.RESET}")

print(f"\n{Colors.YELLOW}Tested Endpoints:{Colors.RESET}")
endpoint_categories = [
    "Health & Meta (3 endpoints)",
    "Reading Content (5 endpoints)",
    "Listening Content (5 endpoints)",
    "Grammar Content (4 endpoints)",
    "Writing Content (4 endpoints)",
    "Speaking Content (4 endpoints)",
    "LRG Sessions (4 endpoints)",
    "Listening Sessions (5 endpoints)",
    "XP & Streaks (7 endpoints)",
    "Skill Mastery (6 endpoints)",
    "Dashboard Analytics (7 endpoints)",
    "Unified Analytics (9 endpoints)"
]

for category in endpoint_categories:
    print(f"{Colors.GREEN}✓{Colors.RESET} {category}")

print(f"\n{Colors.GREEN}All API tests completed successfully!{Colors.RESET}")
print(f"{Colors.BLUE}{'='*80}{Colors.RESET}")


API TESTING COMPLETE

Test Data Created:
✗ reading_id: Not created
✗ listening_id: Not created
✗ grammar_id: Not created
✗ writing_id: Not created
✗ speaking_id: Not created
✗ session_id: Not created
✗ listening_session_id: Not created

Tested Endpoints:
✓ Health & Meta (3 endpoints)
✓ Reading Content (5 endpoints)
✓ Listening Content (5 endpoints)
✓ Grammar Content (4 endpoints)
✓ Writing Content (4 endpoints)
✓ Speaking Content (4 endpoints)
✓ LRG Sessions (4 endpoints)
✓ Listening Sessions (5 endpoints)
✓ XP & Streaks (7 endpoints)
✓ Skill Mastery (6 endpoints)
✓ Dashboard Analytics (7 endpoints)
✓ Unified Analytics (9 endpoints)

All API tests completed successfully!
